In [2]:
import pandas as pd
ad_clicks = pd.read_csv('ad_clicks.csv')

# Analyse Ad Source

Examine the first few rows of data

In [6]:
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


In [5]:
ad_clicks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654 entries, 0 to 1653
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             1654 non-null   object
 1   utm_source          1654 non-null   object
 2   day                 1654 non-null   object
 3   ad_click_timestamp  565 non-null    object
 4   experimental_group  1654 non-null   object
dtypes: object(5)
memory usage: 64.7+ KB


There are 5 variables and 1,654 observations in the `ad_clicks.csv` dataset:
1. user_id: unique user identifier
2. utm_source: ad referral site
3. day: day of the week the ad was served
4. ad_click_timestamp: when user clicked. Null if user did not click on the ad
5. experimental_group: whether test A or B was shown to the user

In [26]:
# Group data by utm_source and do a count
unique_views = ad_clicks.groupby('utm_source').user_id.count().reset_index()
unique_views

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


The ad platform getting the most views is `Google` with **680** views, and followed by `Facebook` with **504** views.

In [32]:
# Using ~ inverts the condition, so if a row is null, it means the user did not click
# A value of True in the is_click column indicates a user clicked on the ad
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

In [35]:
# To know the percent of people who clicked on the ads from each source
# let's group by utm_source and is_click and count the number of user_id‘s in each of those groups
clicks_by_source = ad_clicks.groupby([ 'utm_source', 'is_click' ]).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


The data has been grouped by `utm_source` and by `is_click`. Individual observations of whether a user clicked were counted, but the data looks messy. A **pivot table** is a better representation to understand the number of clicks.

In [36]:
# The columns are is_click (either True or False), 
# the index is utm_source, and the values are user_id.
clicks_pivot = clicks_by_source.pivot(
    columns='is_click',
    index='utm_source',
    values='user_id'
).reset_index()
clicks_pivot

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


With the values arranged in **pivot table** we can now calculate the percentage of users who clicked on the ads.

In [52]:
# Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who 
# clicked on the ad from each utm_source
# Is there a difference in click rates for each source?
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
clicks_pivot

is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


The percentage of users who clicked on ads referred by `Facebook` was about **35.7%**, which is only slightly higher than the clicks from ads referred by `Google` with about **35.1%** of user clicks.

# Analysing A/B Test

In [53]:
# The column experimental_group tells us whether the user was shown Ad A or Ad B.
ad_clicks.groupby('experimental_group').user_id.count().reset_index()

,experimental_group,user_id
0,A,827
1,B,827


Approximately the same number of people were shown both `ad A` and `ad B`.

In [47]:
# We want to check to see if a greater percentage of users clicked on Ad A or Ad B
# so we start by grouping the data by the type of ad, and whether they clicked on it or not (True vs False)
ad_click_percentage = ad_clicks.groupby([ 'experimental_group', 'is_click' ]) \
    .user_id.count() \
    .reset_index()

ad_click_percentage

,experimental_group,is_click,user_id
0,A,False,517
1,A,True,310
2,B,False,572
3,B,True,255


In [48]:
# A pivot table allows to better represent the data
# and it looks cleaner
ad_click_percentage_pivot = ad_click_percentage.pivot(
    columns='is_click',
    index='experimental_group',
    values='user_id'
).reset_index()

ad_click_percentage_pivot

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


The data looks better now, but we still need to calculate the **percentage** of users. To do that, we can create a new column `percent_clicked` like we did earlier.

In [50]:
# Calculate the percentage of users who clicked on ad A and ad B
ad_click_percentage_pivot['percent_clicked'] = ad_click_percentage_pivot[True] / \
    (ad_click_percentage_pivot[True] + ad_click_percentage_pivot[False])

ad_click_percentage_pivot

is_click,experimental_group,False,True,percent_clicked
0,A,517,310,0.374849
1,B,572,255,0.308343


So it seems that **37.4%** of users clicked on `Ad A` as opposed to only **30.8%** of users who clicked on `Ad B`.

In [60]:
# We want to know if the ad clicks have changed by day of the week.
# The first step is creating two new dataframes, that separate the data observations
# according to the type of ad (A vs B)
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

In [65]:
a_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False


In [66]:
b_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True
10,01adb2e7-f711-4ae4-a7c6-29f48457eea1,google,3 - Wednesday,NaN,B,False


In [75]:
#For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.
a_clicks_percent = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()

# Create a pivot table to represent the data better
a_clicks_percent_pivot = a_clicks_percent.pivot(
    columns='is_click',
    index='day',
    values='user_id'
).reset_index()

# Create a new column to calculate the percentage of users who clicked on the ads
# based on the day of the week
a_clicks_percent_pivot['percent_clicked'] = a_clicks_percent_pivot[True] / \
    (a_clicks_percent_pivot[True] + a_clicks_percent_pivot[False])

# And check the results
a_clicks_percent_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,0.380531
1,2 - Tuesday,76,43,0.361345
2,3 - Wednesday,86,38,0.306452
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
5,6 - Saturday,73,45,0.381356
6,7 - Sunday,66,43,0.394495


In [76]:
# And repeat the analysis for ad B
b_clicks_percent = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()

# Create a pivot table to represent the data better
b_clicks_percent_pivot = b_clicks_percent.pivot(
    columns='is_click',
    index='day',
    values='user_id'
).reset_index()

# Create a new column to calculate the percentage of users who clicked on the ads
# based on the day of the week
b_clicks_percent_pivot['percent_clicked'] = b_clicks_percent_pivot[True] / \
    (b_clicks_percent_pivot[True] + b_clicks_percent_pivot[False])

# And check the results
b_clicks_percent_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,0.283186
1,2 - Tuesday,74,45,0.378151
2,3 - Wednesday,89,35,0.282258
3,4 - Thursday,87,29,0.250000
4,5 - Friday,90,38,0.296875
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927


<u>Comparing the results by day of the week and type of A<u>

Based on the click rate of both ads, the company should use `Ad A` because the percentage of user clicks was rather higher when users were shown `Ad A`.
For example, from **Thursday** to **Sunday** users clicked over **38%** of the time. Indeed, **Thursdays** saw a click rate of approximately **40.5%**, which is significantly higher than the same day with `Ad B`.